<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with Pandas pt. 2


---

We've learned about relational databases and the language most use to query them, SQL.  

In this lab we are going to gain more practice converting information to a SQL DB, querying the data and then analyzing it with Python.

In [1]:
# Necessary Libraries
import pandas as pd
import sqlite3
from pandas.io import sql

#### 1.  Read in the EuroMart CSV Data.
- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [2]:
# Reading CSV to Dataframe
orders = pd.read_csv('./datasets/csv/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('./datasets/csv/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('./datasets/csv/EuroMart-SalesTargets.csv', encoding = 'utf-8')

In [3]:
orders.head()

,Order ID,Order Date,Customer Name,City,Country,Region,Segment,Ship Date,Ship Mode,State
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm
1,AZ-2011-9050313,1/3/2011,Summer Hayward,Southport,United Kingdom,North,Consumer,1/7/2011,Economy,England
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,Valence,France,Central,Consumer,1/8/2011,Economy,Auvergne-Rhône-Alpes
3,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England
4,AZ-2011-617423,1/5/2011,Daniel Burke,Echirolles,France,Central,Home Office,1/7/2011,Priority,Auvergne-Rhône-Alpes


In [4]:
OBD.head()

,Order ID,Product Name,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,$854.00,$290.00,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,$140.00,$21.00,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,$27.00,-$22.00,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,$17.00,-$1.00,2,Office Supplies,Storage


In [5]:
sales_targets.head()

,Month of Order Date,Category,Target
0,Jan-11,Furniture,"$10,000.00"
1,Feb-11,Furniture,"$10,100.00"
2,Mar-11,Furniture,"$10,300.00"
3,Apr-11,Furniture,"$10,400.00"
4,May-11,Furniture,"$10,500.00"


#### 2. Rename columns to remove any spaces.

In [6]:
# Rename columns in orders dataframe
orders = orders.rename(columns={'Order ID': 'OrderID', 'Order Date': 'OrderDate', 'Customer Name': 'CustomerName', \
                      'Ship Date': 'ShipDate', 'Ship Mode': 'ShipMode'})
orders.head(1)

,OrderID,OrderDate,CustomerName,City,Country,Region,Segment,ShipDate,ShipMode,State
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [7]:
# Rename columns in OBD dataframe
OBD = OBD.rename(columns={'Order ID': 'OrderID', 'Product Name': 'ProductName'})
OBD.head(1)

,OrderID,ProductName,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper


In [8]:
# Rename columns in sales_targets dataframe
sales_targets = sales_targets.rename(columns={'Month of Order Date': 'MonthOfOrderDate'})
sales_targets.head(1)

,MonthOfOrderDate,Category,Target
0,Jan-11,Furniture,"$10,000.00"


#### 3. Remove dollar signs from sales and profit columns in the order breakdown dataframe.

Convert the columns to float.

In [9]:
OBD.dtypes

OrderID          object
ProductName      object
Discount        float64
Sales            object
Profit           object
Quantity          int64
Category         object
Sub-Category     object
dtype: object

In [10]:
OBD['Sales'] = OBD['Sales'].map(lambda x: x.strip('$'))
OBD['Sales'] = OBD['Sales'].map(lambda x: float(x.replace(',','')))

OBD['Profit'] = OBD['Profit'].map(lambda x: x.replace('$',''))
OBD['Profit'] = OBD['Profit'].map(lambda x: float(x.replace(',','')))

OBD.dtypes

OrderID          object
ProductName      object
Discount        float64
Sales           float64
Profit          float64
Quantity          int64
Category         object
Sub-Category     object
dtype: object

#### 4. Create a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [11]:
# Establishing Local DB connection
db_connection = sqlite3.connect('./datasets/sql/EuroMart.db.sqlite')

In [12]:
# Convert dataframes into SQL databases
orders.to_sql(name='orders', con=db_connection, if_exists='replace', index=False)
OBD.to_sql(name='OBD', con=db_connection, if_exists='replace', index=False)
sales_targets.to_sql(name='sales_targets', con=db_connection, if_exists='replace', index=False)

#### 5. How many orders has each Customer placed? 

In [13]:
sql.read_sql('SELECT CustomerName, \
COUNT(*) AS Total \
FROM orders \
GROUP BY CustomerName \
ORDER BY Total DESC', con=db_connection)

,CustomerName,Total
0,Jose Gambino,13
1,Kayla Tearle,12
2,Mark Washington,12
3,Aaron Bootman,11
4,Georgina Garner,11
5,Hayden Perkins,11
6,Jason Roger,11
7,Jessica Paramor,11
8,Lilly Le Grand,11
9,Lori Miller,11


> *If you're doubting your output check using Pandas*

In [14]:
# Get customer names
customers = sql.read_sql('SELECT CustomerName FROM orders', con=db_connection)

# Count values in list
customers['CustomerName'].value_counts()

Jose Gambino           13
Kayla Tearle           12
Mark Washington        12
Georgina Garner        11
Hayden Perkins         11
Lori Miller            11
Lilly Le Grand         11
Jessica Paramor        11
Jason Roger            11
Rachel Tyler           11
Michael Myers          11
Patricia Smith         11
Maya Pamphlett         11
Aaron Bootman          11
Terence Welch          11
Michael Watkins        10
Ngoc Orozco            10
Rory Jess              10
Philip Newsom          10
Chelsea Bannister      10
Jasmine Slater         10
Elise Ali              10
Robert Quick           10
Angie Massengill       10
Elijah Sodeman         10
Eloise Sykes           10
Katie Evenden          10
Kian Sykes             10
Peter Daly             10
Alexandra Macdonald    10
                       ..
Jay Collier             1
Erin Wallis             1
Grace Brown             1
Alexandra Mahmood       1
James Brown             1
Imogen Smithies         1
Claire Dilke            1
Patrick Atki

#### 6. Create a Query to return a table of only geographic features from the List of Orders Table.

In [15]:
sql.read_sql('SELECT City, Country, Region, State FROM orders', con=db_connection)

,City,Country,Region,State
0,Stockholm,Sweden,North,Stockholm
1,Southport,United Kingdom,North,England
2,Valence,France,Central,Auvergne-Rhône-Alpes
3,Birmingham,United Kingdom,North,England
4,Echirolles,France,Central,Auvergne-Rhône-Alpes
5,La Seyne-sur-Mer,France,Central,Provence-Alpes-Côte d'Azur
6,Toulouse,France,Central,Languedoc-Roussillon-Midi-Pyrénées
7,Genoa,Italy,South,Liguria
8,Vienna,Austria,Central,Vienna
9,Murcia,Spain,South,Murcia


#### 7. Create a Query to return a table with all of the orders that had a negative profit from the Order Breakdown Table.

In [16]:
sql.read_sql('SELECT * FROM OBD WHERE Profit < 0', con=db_connection).head()

,OrderID,ProductName,Discount,Sales,Profit,Quantity,Category,Sub-Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


#### 8. Construct a query to return a table with the Customer Name and Product Name.  

> **Note:** This will require a join!

In [17]:
sql.read_sql('SELECT orders."OrderID", orders."CustomerName", OBD."ProductName" \
FROM orders \
INNER JOIN OBD \
ON orders."OrderID" = OBD."OrderID"', con=db_connection)

,OrderID,CustomerName,ProductName
0,BN-2011-7407039,Ruby Patel,"Enermax Note Cards, Premium"
1,AZ-2011-9050313,Summer Hayward,"Dania Corner Shelving, Traditional"
2,AZ-2011-6674300,Devin Huddleston,"Binney & Smith Sketch Pad, Easy-Erase"
3,BN-2011-2819714,Mary Parker,"Boston Markers, Easy-Erase"
4,BN-2011-2819714,Mary Parker,"Eldon Folders, Single Width"
5,AZ-2011-617423,Daniel Burke,"Binney & Smith Pencil Sharpener, Water Color"
6,AZ-2011-617423,Daniel Burke,"Sanford Canvas, Fluorescent"
7,AZ-2011-2918397,Fredrick Beveridge,"Accos Thumb Tacks, Assorted Sizes"
8,AZ-2011-2918397,Fredrick Beveridge,"Bush Floating Shelf Set, Pine"
9,AZ-2011-2918397,Fredrick Beveridge,"Smead Lockers, Industrial"


#### 9.  How many orders for "Office Supplies" (Category) has Sweden made?

> **Note:** from this point on you'll probably be combining SQL and Pandas, in that you would use SQL querys to gather the relevant information and use Pandas to analyze it.

In [18]:
sweden_offsupp = sql.read_sql('SELECT orders."OrderID", orders."Country", OBD."Category" \
FROM orders \
LEFT JOIN OBD \
ON orders."OrderID" = OBD."OrderID" \
WHERE orders."Country" = "Sweden" and OBD."Category" = "Office Supplies"', con=db_connection)

sweden_offsupp.count()

OrderID     133
Country     133
Category    133
dtype: int64

#### 10. What was the total sales for products that have been discounted? 

In [20]:
sales_disc = sql.read_sql('SELECT Discount, Sales FROM OBD WHERE Discount > 0', con=db_connection)
sales_disc['Sales'].sum()

1115614.0

#### 11. What is the total quantity of objects sold for each country?

In [23]:
quant_country = sql.read_sql('SELECT OBD."Quantity", orders."Country" \
FROM orders \
INNER JOIN OBD \
ON OBD."OrderID" = orders."OrderID"', con=db_connection)

quant_country.groupby('Country').sum()

,Quantity
Country,
Austria,973
Belgium,532
Denmark,204
Finland,201
France,7329
Germany,6179
Ireland,392
Italy,3612
Netherlands,1526


#### 12. In what Countries are profits lowest? (Report lowest 5-10)

In [38]:
profits = sql.read_sql('SELECT orders."Country", OBD."Profit" \
FROM orders \
INNER JOIN OBD \
ON OBD."OrderID" = orders."OrderID"', con=db_connection)

profits.groupby('Country').sum().sort_values('Profit').reset_index()[5:11]

,Country,Profit
5,Finland,3908.0
6,Norway,5167.0
7,Switzerland,7234.0
8,Belgium,9912.0
9,Italy,15802.0
10,Austria,21332.0


#### 13. What Counties have the best and worst Sales to Profit Ratios?
(Total Sales divided by Total Profits.)
Essentially this is saying for every dollar of product sold, how much is profit.

In [42]:
sp_ratio = sql.read_sql('SElECT orders."Country", OBD."Sales", OBD."Profit" \
FROM orders \
INNER JOIN OBD \
ON OBD."OrderID" = orders."OrderID"', con=db_connection) 

sp_ratio2 = sp_ratio.groupby('Country').sum().sort_values('Profit')

sp_ratio2['SalesProfitRatio'] = sp_ratio2['Profit'] / sp_ratio2['Sales']

sp_ratio2.sort_values('SalesProfitRatio', ascending=False)

# Switzerland has the best, Portugal has the worst

,Sales,Profit,SalesProfitRatio
Country,,,
Switzerland,24874.0,7234.0,0.290826
Austria,79382.0,21332.0,0.268726
Norway,20529.0,5167.0,0.251693
Belgium,42320.0,9912.0,0.234216
United Kingdom,420497.0,90382.0,0.214941
Finland,20702.0,3908.0,0.188774
Spain,249402.0,47067.0,0.188719
Germany,488681.0,86279.0,0.176555
France,609683.0,70067.0,0.114924


#### 14. What Shipping method is most common for 'Bookcases' (Sub Category)?

In [48]:
sql.read_sql('SELECT orders."ShipMode", OBD."Sub-Category" \
FROM orders \
INNER JOIN OBD \
ON OBD."OrderID" = orders."OrderID" \
WHERE OBD."Sub-Category" = "Bookcases"', con=db_connection)['ShipMode'].value_counts()

# Economy

Economy         234
Economy Plus     76
Priority         59
Immediate        22
Name: ShipMode, dtype: int64

#### 15 .What city in the Orders table generated the highest net sales?  (List all the cities and countries in descending order by net sales.)

In [49]:
sql.read_sql('SELECT orders."City", orders."Country", OBD."Sales" \
FROM orders \
INNER JOIN OBD \
ON orders."OrderID" = OBD."OrderID"', \
             con=db_connection).groupby(['City', 'Country']).sum().sort_values('Sales', ascending=False)

# London, UK

,,Sales
City,Country,
London,United Kingdom,69230.0
Berlin,Germany,52555.0
Vienna,Austria,51844.0
Madrid,Spain,44981.0
Paris,France,42245.0
Rome,Italy,28330.0
Barcelona,Spain,27405.0
Hamburg,Germany,23574.0
Marseille,France,21677.0


#### BONUS: Create a Column called 'Shipping Delay' on the 'orders' table, which is the difference in days between 'Order Date' and 'Ship Date'.

In [50]:
# Check datatypes
orders.dtypes

OrderID         object
OrderDate       object
CustomerName    object
City            object
Country         object
Region          object
Segment         object
ShipDate        object
ShipMode        object
State           object
dtype: object

In [52]:
# Convert datatype from string to datetime
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
orders['ShipDate'] = pd.to_datetime(orders['ShipDate'])

In [53]:
# Re-check datatypes to confirm the conversion worked
orders.dtypes

OrderID                 object
OrderDate       datetime64[ns]
CustomerName            object
City                    object
Country                 object
Region                  object
Segment                 object
ShipDate        datetime64[ns]
ShipMode                object
State                   object
dtype: object

In [59]:
orders['ShippingDelay'] = (orders['ShipDate'] - orders['OrderDate']).astype('timedelta64[D]')
orders.head()

,OrderID,OrderDate,CustomerName,City,Country,Region,Segment,ShipDate,ShipMode,State,ShippingDelay
0,BN-2011-7407039,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,4.0
1,AZ-2011-9050313,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,4.0
2,AZ-2011-6674300,2011-01-04,Devin Huddleston,Valence,France,Central,Consumer,2011-01-08,Economy,Auvergne-Rhône-Alpes,4.0
3,BN-2011-2819714,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,5.0
4,AZ-2011-617423,2011-01-05,Daniel Burke,Echirolles,France,Central,Home Office,2011-01-07,Priority,Auvergne-Rhône-Alpes,2.0


#### BONUS: Update your Orders table in your Sqlite DB to include the 'Shipping Delay' feature.

In [60]:
orders.to_sql(name='orders', con=db_connection, if_exists='replace', index=False)

#### BONUS: Which Product Category has the highest average 'Shipping Delay'?

In [61]:
sql.read_sql('SELECT OBD."Category", orders."ShippingDelay" \
FROM OBD \
INNER JOIN orders \
ON orders."OrderID" = OBD."OrderID" \
', con=db_connection).groupby('Category').mean()

,ShippingDelay
Category,
Furniture,4.000000
Office Supplies,3.975028
Technology,4.125410


### Challenge problem:   
**In what months and Categories were Sales Targets Exceeded?**

---

This may require a considerable amount of data processing.

In [22]:
# A: